In [5]:
import win32com.client
import time

Vissim = win32com.client.Dispatch("Vissim.Vissim")

net_path = r"C:\VISSIM_Workspace\화성~서울\ToyNetwork\test.inpx"
lay_path = r"C:\VISSIM_Workspace\화성~서울\ToyNetwork\test.layx"

TIME_FROM = 100
TIME_TO = 200

Vissim.LoadNet(net_path)
Vissim.LoadLayout(lay_path)
time.sleep(1)

# =================================================
# 시뮬레이션 기본 설정 (필수)
# =================================================
sim = Vissim.Simulation
sim.SetAttValue("SimRes", 10)
sim.SetAttValue("SimPeriod", 8400)   # TIME_TO보다 크게
sim.SetAttValue("RandSeed", 9811872)


rsas = Vissim.Net.ReducedSpeedAreas.GetAll()
stop_rsas = [rsa for rsa in rsas if rsa.AttValue("Name") == "정지"]

# RSA 시간 설정
for rsa in stop_rsas:
    rsa.SetAttValue("TimeTo", TIME_TO)
    rsa.SetAttValue("TimeFrom", TIME_FROM)

prev_mode = None
last_t = -1

while True:
    t = sim.AttValue("SimSec")

    # Reset 감지
    if t < last_t:
        print("⚠ SimSec reset detected.")
        break

    # 정상 종료
    if t >= sim.AttValue("SimPeriod"):
        break

    last_t = t


    sim.RunSingleStep()

print(f"✅ Scenario finished | TIME_TO = {TIME_TO}")

# 종료
Vissim.Exit()
print("\n🎉 All scenarios completed.")

com_error: (-2147418111, '피호출자가 호출을 거부했습니다.', None, None)